In [ ]:
!pip install tensorflow

# 모델 불러오기 및 기본 함수

In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras

In [ ]:
# 모델 불러오기
model = keras.models.load_model('final_model_7931.h5')

# 예측 확률을 계산하는 함수 (final_model.h5 모델 사용)
def calculate_probability(ratios):
    # ratios는 (6,) 형태이므로 모델 입력 형태에 맞게 (1, 6, 1) 형태로 reshape
    input_data = np.array(ratios).reshape(1, 6, 1)
    # 모델 예측 결과
    probability = model.predict(input_data)[0][0]
    return probability

# 최소 비율을 유지하며 합이 1이 되도록 정규화하는 함수
def normalize_ratios(ratios, min_ratio=0.09):
    # 각 비율이 최소 min_ratio 이상이 되도록 클리핑
    ratios = np.maximum(ratios, min_ratio)

    # 비율 합이 1이 되도록 정규화
    ratios = ratios / np.sum(ratios)

    # 정규화 후에도 비율이 min_ratio 이상이 되도록 조정 반복
    while np.any(ratios < min_ratio):
        deficit = np.sum(np.maximum(min_ratio - ratios, 0))
        ratios = np.where(ratios >= min_ratio, ratios, min_ratio)
        ratios = ratios / np.sum(ratios)  # 다시 정규화하여 합이 1이 되도록 조정

    return ratios

# Simulated Annealing (시뮬레이티드 어닐링)

### 적용 방법:
- 초기 상태 설정: 비율 조합의 초기 상태를 무작위로 설정
- 변형 적용: 매 반복마다 새로운 비율 조합 생성, 확률적으로 조합 선택
- 점진적 안정화: 반복을 거듭할수록 변동 범위를 줄여 최적의 조합에 수렴하도록


In [ ]:
import numpy as np
import random
from tensorflow.keras.models import load_model

# 시뮬레이티드 어닐링 함수 정의
def simulated_annealing(initial_ratios, temperature, cooling_rate, iterations):
    # 초기 설정
    current_ratios = initial_ratios
    current_score = calculate_probability(current_ratios)
    best_ratios = current_ratios
    best_score = current_score

    for i in range(iterations):
        # 현재 온도 설정
        temp = temperature / (1 + cooling_rate * i)

        # 새 비율 조합 생성
        new_ratios = current_ratios + np.random.normal(0, 0.05, size=len(initial_ratios))
        new_ratios = normalize_ratios(new_ratios)  # 정규화 및 최소 비율 적용

        # 새 조합의 점수 계산
        new_score = calculate_probability(new_ratios)

        # 점수 비교 및 업데이트
        if new_score > current_score or random.uniform(0, 1) < np.exp((new_score - current_score) / temp):
            current_ratios = new_ratios
            current_score = new_score

            # 최고 점수 갱신
            if new_score > best_score:
                best_ratios = new_ratios
                best_score = new_score

    return best_ratios, best_score

# 초기 파라미터 설정
initial_ratios = np.array([0.2, 0.2, 0.2, 0.2, 0.1, 0.1])  # 클러스터 비율 초기값
temperature = 10.0  # 초기 온도
cooling_rate = 0.01  # 냉각률
iterations = 2000  # 반복 횟수

# 시뮬레이티드 어닐링 실행
best_ratios, best_score = simulated_annealing(initial_ratios, temperature, cooling_rate, iterations)

# 결과 출력
print("최적의 비율 조합:", best_ratios)
print("최적의 예측 확률:", best_score)
